# Model selection

#### Import packages

In [1]:
!pip install pandas
!pip install torch torchvision torchaudio
!pip install pandas
!pip install transformers
!pip install scikit-learn
!pip install torch
!pip install tensorflow
!pip install --upgrade pip
!pip install --upgrade numpy
!pip install --upgrade transformers
!pip install --upgrade tensorflow
!pip install datasets 

import tensorflow as tf
import pandas as pd

from transformers import pipeline
from datasets import load_dataset
from transformers import pipeline
from sklearn.metrics import accuracy_score, f1_score

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.2.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0

2025-01-02 18:52:29.999567: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735840350.027002    4066 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735840350.035537    4066 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-02 18:52:30.067627: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Model selection for dataset 1
##### Twitter dataset: https://huggingface.co/datasets/dair-ai/emotion 

### Load Twitter dataset  6 basic emotions (['fear' 'sad' 'love' 'joy' 'suprise' 'anger'])

In [2]:
ds = load_dataset("dair-ai/emotion", "split")

README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [3]:
# Preprocess Twitter dataset
print(ds)
# Select the train part
df_2 = ds['train']
train_df = pd.DataFrame(df_2)
label_to_emotion = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}

# Create the 'emotion' column by mapping 'label' to 'emotion'
train_df['emotion'] = train_df['label'].map(label_to_emotion)
df_1 = train_df[["text", "emotion"]]

# Select only 10000 rows
df_1 = df_1.iloc[:10000]
print(df_1)
print(df_1['emotion'].unique())

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})
                                                   text  emotion
0                               i didnt feel humiliated  sadness
1     i can go from feeling so hopeless to so damned...  sadness
2      im grabbing a minute to post i feel greedy wrong    anger
3     i am ever feeling nostalgic about the fireplac...     love
4                                  i am feeling grouchy    anger
...                                                 ...      ...
9995  i have a desk job and sit on my ass all day lo...     fear
9996     i feel like such a confused person lately sigh     fear
9997  i feel ive been beaten down by the words of me...  sadness
9998  i fully understand the feeling of being beaten...  sadness

## Model 1 bert-base-uncased-emotion

In [4]:
# The BERT classifier 
classifier = pipeline("text-classification",
                      model='bhadresh-savani/bert-base-uncased-emotion', 
                      return_all_scores=True,
                      framework="pt")

                      
# Predictions on all sentences in the dataset
predictions = []
for text in df_1['text']:
    result = classifier(text)  # Returns a list of dictionaries
    predicted_class = max(result[0], key=lambda x: x['score'])['label']
    predictions.append(predicted_class)

# Compute accuracy
accuracy = accuracy_score(df_1['emotion'], predictions)

# Compute F1 score
f1 = f1_score(df_1['emotion'], predictions, average='weighted') 

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0
/home/ucloud/.local/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Accuracy: 0.9837
F1 Score: 0.9837


## Model 2 roberta-base-emotion

In [5]:
# The classifier 
classifier = pipeline("text-classification",
                      model='bhadresh-savani/roberta-base-emotion',
                      return_all_scores=True,
                      framework="pt"
                      )
                      
# Predictions on all sentences in the dataset
predictions = []
for text in df_1['text']:
    result = classifier(text)  # Returns a list of dictionaries
    
    predicted_class = max(result[0], key=lambda x: x['score'])['label']
    predictions.append(predicted_class)

# Compute accuracy
accuracy = accuracy_score(df_1['emotion'], predictions)

# Compute F1 score
f1 = f1_score(df_1['emotion'], predictions, average='weighted') 

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

config.json:   0%|          | 0.00/983 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0
/home/ucloud/.local/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Accuracy: 0.9555
F1 Score: 0.9552


## Model 3  distilbert-base-uncased-emotion

In [6]:
# The classifier 
classifier = pipeline("text-classification",
                      model='bhadresh-savani/distilbert-base-uncased-emotion',
                      return_all_scores=True,
                      framework="pt"
                      )
                      
# Predictions on all sentences in the dataset
predictions = []
for text in df_1['text']:
    result = classifier(text)  # Returns a list of dictionaries
    
    predicted_class = max(result[0], key=lambda x: x['score'])['label']
    predictions.append(predicted_class)

# Compute accuracy
accuracy = accuracy_score(df_1['emotion'], predictions)

# Compute F1 score
f1 = f1_score(df_1['emotion'], predictions, average='weighted') 

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0
/home/ucloud/.local/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Accuracy: 0.9877
F1 Score: 0.9877


# Model selection for dataset 2
##### https://www.kaggle.com/datasets/kushagra3204/sentiment-and-emotion-analysis-dataset 

### Load dataset with 6 basic emotions (['fear' 'sad' 'love' 'joy' 'suprise' 'anger'])

In [7]:
data = pd.read_csv('/work/VNV/Module 2 - Model selection/data/combined_emotion.csv') 
#Select relevant columns
data = data[["sentence", "emotion"]]
print(data)
df = data.iloc[:10000]

# Replace 'sad' with 'sadness' in the 'emotion' column using .loc
df.loc[:, 'emotion'] = df['emotion'].replace('sad', 'sadness')

print(df['emotion'].unique())

                                                 sentence emotion
0           i just feel really helpless and heavy hearted    fear
1       ive enjoyed being able to slouch about relax a...     sad
2       i gave up my internship with the dmrg and am f...    fear
3                              i dont know i feel so lost     sad
4       i am a kindergarten teacher and i am thoroughl...    fear
...                                                   ...     ...
422741                 i begun to feel distressed for you    fear
422742  i left feeling annoyed and angry thinking that...   anger
422743  i were to ever get married i d have everything...     joy
422744  i feel reluctant in applying there because i w...    fear
422745  i just wanted to apologize to you because i fe...   anger

[422746 rows x 2 columns]
['fear' 'sadness' 'love' 'joy' 'suprise' 'anger']


## Model 1 bert-base-uncased-emotion

In [8]:
# The BERT classifier 
classifier = pipeline("text-classification",
                      model='bhadresh-savani/bert-base-uncased-emotion', 
                      return_all_scores=True,
                      framework="pt")

                      
# Predictions on all sentences in the dataset
predictions = []
for sentence in df['sentence']:
    result = classifier(sentence)  # Returns a list of dictionaries
    
    predicted_class = max(result[0], key=lambda x: x['score'])['label']
    predictions.append(predicted_class)

# Compute accuracy
accuracy = accuracy_score(df['emotion'], predictions)

# Compute F1 score
f1 = f1_score(df['emotion'], predictions, average='weighted') 

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Device set to use cuda:0
/home/ucloud/.local/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Accuracy: 0.9098
F1 Score: 0.9090


## Model 2 roberta-base-emotion

In [9]:
# The classifier 
classifier = pipeline("text-classification",
                      model='bhadresh-savani/roberta-base-emotion',
                      return_all_scores=True,
                      framework="pt"
                      )
                      
# Predictions on all sentences in the dataset
predictions = []
for sentence in df['sentence']:
    result = classifier(sentence)  # Returns a list of dictionaries
    
    predicted_class = max(result[0], key=lambda x: x['score'])['label']
    predictions.append(predicted_class)

# Compute accuracy
accuracy = accuracy_score(df['emotion'], predictions)

# Compute F1 score
f1 = f1_score(df['emotion'], predictions, average='weighted') 

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

Device set to use cuda:0
/home/ucloud/.local/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Accuracy: 0.9104
F1 Score: 0.9056


## Model 3  distilbert-base-uncased-emotion

In [ ]:
# The classifier 
classifier = pipeline("text-classification",
                      model='bhadresh-savani/distilbert-base-uncased-emotion',
                      return_all_scores=True,
                      framework="pt"
                      )
                      
# Predictions on all sentences in the dataset
predictions = []
for sentence in df['sentence']:
    result = classifier(sentence)  # Returns a list of dictionaries
    
    predicted_class = max(result[0], key=lambda x: x['score'])['label']
    predictions.append(predicted_class)

# Compute accuracy
accuracy = accuracy_score(df['emotion'], predictions)

# Compute F1 score
f1 = f1_score(df['emotion'], predictions, average='weighted') 

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
/home/ucloud/.local/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Accuracy: 0.9089
F1 Score: 0.9070


# Creating table with the results

In [ ]:
# Data for the table
data = {
    "Model": ["Model 1 -bert-base-uncased emotion", "Model 2 -roberta-base-emotion", "Model 3 -distilbert-base-uncased-emotion"],
    "Twitter dataset - Accuracy": [98.37, 95.55, 98.77],
    "Twitter dataset - F1 Score": [98.37, 95.52, 98.77],
    "Kaggle dataset - Accuracy": [90.98, 91.04, 90.89],
    "Kaggle dataset - F1 Score": [90.90, 90.56, 90.70],
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv("/work/VNV/Module 2 - Model selection/data/model_evaluation.csv", index=False)
df

,Model,Twitter dataset - Accuracy,Twitter dataset - F1 Score,Kaggle dataset - Accuracy,Kaggle dataset - F1 Score
0,Model 1 -bert-base-uncased emotion,98.37,98.37,90.98,90.90
1,Model 2 -roberta-base-emotion,95.55,95.52,91.04,90.56
2,Model 3 -distilbert-base-uncased-emotion,98.77,98.77,90.89,90.70
